# Tensor product in quantum computing

In [2]:
import numpy as np

In [3]:
_0 = np.expand_dims((np.array([1, 0])), 1,)
_0

array([[1],
       [0]])

In [4]:
_1 = np.expand_dims((np.array([0, 1])), 1,)
_1

array([[0],
       [1]])

In [5]:
(_1 @ _1.T).flatten()

array([0, 0, 0, 1])

In [6]:
from itertools import accumulate, product, repeat
from functools import reduce
from operator import mul

def bouter(m1, m2):
    """
    Binary (in sense of parity of operation!) outer product of two
    column vectors.
    """
    return (m1 @ m2.T)[..., None]

def bbouter(m1, m2):
    """
    Better Binary (in sense of parity of operation!) outer product of two
    arbitrary tensors, i.e. even matrices.
    """
    return np.squeeze(np.tensordot(m1, m2, axes=0))

def outer(*ms):
    """
    Outer product of multiple tensors, uses bbouter.
    """
    return reduce(bbouter, ms)

def verbose_outer(*ms):
    """
    Returns all intermediate results.
    """
    return accumulate(ms, bbouter)

def mat_outer(*ms):
    """
    Outer product which "flattens" output into appropriate matrix.
    """
    return flat_mat(outer(*ms))

def mat_outer_pow(m, e: int):
    return mat_outer(*repeat(m, e))

def col_outer(*ms):
    """
    Outer product which "flattens" result into column vector.
    """
    return flat_col(outer(*ms))

def col_outer_pow(m, e: int):
    return col_outer(*repeat(m, e))

def flat_col(m):
    return m.flatten()[..., None]

def flat_mat(m):
    """
    Flattens tensor product of single qubit transforms 
    matrices to shape 2^n x 2^n, where n is the number 
    of qubits.
    """
    n = int(np.sqrt(m.size))
    odds = np.arange(m.ndim, dtype=int)[1:-1:2]
    evens = np.arange(m.ndim, dtype=int)[2:-1:2]
    # MAGIC :-)
    return m.transpose(0, *evens, *odds, m.ndim - 1).reshape(n, n)


# Display utils
def str01(m):
    """
    _0 = [1, 0]^T or _1 = [0, 1]^T vector to string "0" or "1"
    """
    if bool(m.T @ _0):
        return "0"
    else:
        return "1"

def strket(*m):
    """
    String ket representation of sequence of _0/_1 vectors.
    """
    return f"|{''.join(map(str01, m))}>"

In [7]:
bbouter(_0, _0)

array([[1, 0],
       [0, 0]])

In [8]:
list(map(lambda s: s.shape, accumulate([_0, _1, _1, _0], bbouter)))

[(2, 1), (2, 2), (2, 2, 2), (2, 2, 2, 2)]

In [9]:
col_outer(_1, _1, _1)

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1]])

## All eigen states of n qubit sytem

In [10]:
n = 3  # number of qubits
eigenstates = {strket(*s): col_outer(*s) for s in  product([_0, _1], repeat=n)}

In [11]:
eigenstates

{'|000>': array([[1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0]]), '|001>': array([[0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0]]), '|010>': array([[0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0]]), '|011>': array([[0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0]]), '|100>': array([[0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0]]), '|101>': array([[0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0]]), '|110>': array([[0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0]]), '|111>': array([[0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1]])}

In [12]:
eigenkets = {str(v.flatten()): k for k, v in eigenstates.items()}
eigenkets

{'[1 0 0 0 0 0 0 0]': '|000>',
 '[0 1 0 0 0 0 0 0]': '|001>',
 '[0 0 1 0 0 0 0 0]': '|010>',
 '[0 0 0 1 0 0 0 0]': '|011>',
 '[0 0 0 0 1 0 0 0]': '|100>',
 '[0 0 0 0 0 1 0 0]': '|101>',
 '[0 0 0 0 0 0 1 0]': '|110>',
 '[0 0 0 0 0 0 0 1]': '|111>'}

##  Eigenstates mixture
Requires Sage kernel!

In [13]:
cs = var(",".join([f"c_{i}" for i in range(2**n)]))
cs

(c_0, c_1, c_2, c_3, c_4, c_5, c_6, c_7)

In [15]:
phi = sum([c * state for c, state in zip(cs, eigenstates.values())])

In [16]:
phi

array([[c_0],
       [c_1],
       [c_2],
       [c_3],
       [c_4],
       [c_5],
       [c_6],
       [c_7]], dtype=object)

## Quantum gates

In [17]:
# single qubit Hadamard gate
Had = 1/sqrt(2) * np.array([[1, 1],
                            [1, -1]])

In [18]:
Had @ _0

array([[1/2*sqrt(2)],
       [1/2*sqrt(2)]], dtype=object)

In [19]:
# two qubit gate from hadamard and identity
m1 = mat_outer(Had, np.eye(2))
m1

array([[0.5*sqrt(2), 0.0, 0.5*sqrt(2), 0.0],
       [0.0, 0.5*sqrt(2), 0.0, 0.5*sqrt(2)],
       [0.5*sqrt(2), 0.0, -0.5*sqrt(2), -0.0],
       [0.0, 0.5*sqrt(2), -0.0, -0.5*sqrt(2)]], dtype=object)

In [20]:
m1.shape

(4, 4)

In [21]:
# applying gate to state |10> produces state sqrt(2)/2*|00>  - sqrt(2)/2*|10>
m1 @ col_outer(_1, _0)

array([[0.5*sqrt(2)],
       [0],
       [-0.5*sqrt(2)],
       [0]], dtype=object)

In [25]:
eigenstates["|100>"]

array([[0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0]])

In [24]:
# state of 3 qubits with first qubit in superposition and rest zeroed
col_outer((1/sqrt(2) * _0 - 1/sqrt(2) * _1), _0,  _0)

array([[1/2*sqrt(2)],
       [0],
       [0],
       [0],
       [-1/2*sqrt(2)],
       [0],
       [0],
       [0]], dtype=object)

In [26]:
# this gate produces same result as above
m2 = outer(Had, 
           np.eye(2), 
           np.eye(2))
m2.shape

(2, 2, 2, 2, 2, 2)

In [27]:
# transpose and reshape is a bit hairy due to hierarchical structure of tensor product
m2.transpose(0, 2, 4, 1, 3, 5).reshape(8, 8) @ col_outer(_1, _0, _0)

array([[0.5*sqrt(2)],
       [0],
       [0],
       [0],
       [-0.5*sqrt(2)],
       [0],
       [0],
       [0]], dtype=object)

In [28]:
# transpose and reshape hidden in flat_mat
flat_mat(m2) @ col_outer(_1, _0, _0)

array([[0.5*sqrt(2)],
       [0],
       [0],
       [0],
       [-0.5*sqrt(2)],
       [0],
       [0],
       [0]], dtype=object)

In [29]:
# mat_outer does everything automatically, also different gate works too!
mat_outer(np.eye(2), 
          Had, 
          np.eye(2)) @ col_outer(_1, _0, _0)

array([[0],
       [0],
       [0],
       [0],
       [0.5*sqrt(2)],
       [0],
       [0.5*sqrt(2)],
       [0]], dtype=object)

In [30]:
1/sqrt(2) * eigenstates["|110>"] + 1/sqrt(2) * eigenstates["|100>"]

array([[0],
       [0],
       [0],
       [0],
       [1/2*sqrt(2)],
       [0],
       [1/2*sqrt(2)],
       [0]], dtype=object)

In [31]:
mat_outer(Had, Had, Had) @ col_outer(_1, _0, _0)

array([[1/4*sqrt(2)],
       [1/4*sqrt(2)],
       [1/4*sqrt(2)],
       [1/4*sqrt(2)],
       [-1/4*sqrt(2)],
       [-1/4*sqrt(2)],
       [-1/4*sqrt(2)],
       [-1/4*sqrt(2)]], dtype=object)

In [32]:
# Swap gate yayy!
Swap = np.array([[1, 0, 0, 0],
                 [0, 0, 1, 0],
                 [0, 1, 0, 0],
                 [0, 0, 0, 1]])

In [33]:
Swap @ col_outer(_0, _1)

array([[0],
       [0],
       [1],
       [0]])

In [34]:
Swap @ col_outer(Had @ _0, _0)

array([[1/2*sqrt(2)],
       [1/2*sqrt(2)],
       [0],
       [0]], dtype=object)

In [35]:
col_outer(col_outer(Had @ _0, _0), _0)

array([[1/2*sqrt(2)],
       [0],
       [0],
       [0],
       [1/2*sqrt(2)],
       [0],
       [0],
       [0]], dtype=object)

In [36]:
eigenstates["|100>"]

array([[0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0]])

In [37]:
col_outer_pow(col_outer(_0, _1), 2) == col_outer(_0, _1, _0, _1)

array([[ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True]])